In [1]:
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from random import shuffle
import os
from keras.models import Model, Sequential, load_model
from keras import layers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D, ZeroPadding2D
from keras import applications
from keras.utils import plot_model, to_categorical
from keras.applications import VGG16, ResNet50
from keras.optimizers import RMSprop,Adam    
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import cv2

Using TensorFlow backend.


In [2]:
filenames = os.listdir("train")
shuffle(filenames)
age=[]  #Children (1-14) CLASS 0;Youth (14-25) CLASS 1;ADULTS (25-40) CLASS 2;Middle age (40-60) CLASS 3
         #Very Old (>60) CLASS 4
gender=[] #0=male ;1=female

for filename in filenames:
    label_1=filename.split('_')[0]
    label_1 = int(label_1)
    if label_1 <= 14:
        age.append(str(0))
    if (label_1>14) and (label_1<=25):
        age.append(str(1))
    if (label_1>25) and (label_1<40):
        age.append(str(2))
    if (label_1>=40) and (label_1<60):
        age.append(str(3))
    if label_1>=60:
        age.append(str(4))
   
for filename in filenames:
    label_2=filename.split('_')[1]
    label_2 = int(label_2)
    if label_2 == 0:
        gender.append(str(0))
    if label_2 == 1:
        gender.append(str(1))    

In [3]:
df=pd.DataFrame({
    'filename':filenames,
    'age':age,
    'gender':gender
})
df.head()

,filename,age,gender
0,8_1_0_20170109204542015.jpg.chip.jpg,0,1
1,1_1_2_20161219201453804.jpg.chip.jpg,0,1
2,18_0_3_20170104230349313.jpg.chip.jpg,1,0
3,22_1_3_20170119163028157.jpg.chip.jpg,1,1
4,26_1_1_20170112213007903.jpg.chip.jpg,2,1


In [4]:
img_row = 32
img_col = 32
epochs = 30
batch_size = 128

In [5]:
pre_trained_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_row,img_col,3)) #taking pretrained weights;top layer chopped off as i am going to use it of my own;
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False  

for layer in pre_trained_model.layers[15:]:
    layer.trainable = False 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [6]:
top_model = pre_trained_model.output #output of vgg16 last layer taken as input here
top_model = Flatten()(top_model)
layer_1 = Dense(1024,activation='relu')(top_model)
layer_2= Dense(1024,activation='relu')(top_model)
#layer_1 = Dense(128,activation='relu')(layer_1)
layer_1 = Dropout(0.5)(layer_1)
layer_2 = Dropout(0.5)(layer_2)
out_1 = Dense(5,activation='softmax',name='out_1')(layer_1) #for age
out_2 = Dense(1,activation='sigmoid',name='out_2')(layer_2) #for gender




In [7]:
model = Model(inputs = pre_trained_model.input ,outputs=out_2) #use out_1 for age and out_2 for gender

In [8]:
train_df, validate_df = train_test_split(df, test_size=0.1)#df is the dataframe;train_test_split er prothom
train_df = train_df.reset_index()                #arguement ta has to be a pd.dataframe,np.array or list
validate_df = validate_df.reset_index() #ekdom shamner index gula 0 theke shuru hoi age jai thakuk
                                        #test_size determines kottuk test set e thakbe
# validate_df = validate_df.sample(n=100).reset_index() # use for fast testing code purpose
# train_df = train_df.sample(n=1800).reset_index() # use for fast testing code purpose

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

print(total_train)
print(total_validate)

21337
2371


In [9]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "train", 
    x_col='filename',
    y_col='gender',
    class_mode='binary',                    #'multi_output'],
    target_size=(img_row, img_col),
    batch_size=batch_size
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "train", #duitai training file theke kintu
    x_col='filename',
    y_col='gender',
    class_mode='binary',
    target_size=(img_row, img_col),
    batch_size=batch_size
)


Found 21337 validated image filenames belonging to 2 classes.
Found 2371 validated image filenames belonging to 2 classes.


In [15]:
model.compile(optimizer=Adam(lr=.0001),                              #{'out_1':'adam','out_2':'adam'}, 
              loss='binary_crossentropy',
              metrics =['accuracy']                           #{'out_1':"accuracy",'out_2':"accuracy"}
             )

In [12]:
from keras.optimizers import RMSprop,Adam    #import activation function that you need
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau 



learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', #helps to monitor if val_loss;if it doesn't reduce after 5 epochs, the learning rate will be decreased and checked again in the same way;this will be done upto lr=.001
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks=[learning_rate_reduction]

In [13]:
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size)


Epoch 1/30
166/166 [==============================] - 65s 394ms/step - loss: 0.5857 - accuracy: 0.6929 - val_loss: 0.5464 - val_accuracy: 0.7517
Epoch 2/30
166/166 [==============================] - 68s 409ms/step - loss: 0.5530 - accuracy: 0.7178 - val_loss: 0.5205 - val_accuracy: 0.7570
Epoch 3/30
166/166 [==============================] - 68s 412ms/step - loss: 0.5365 - accuracy: 0.7323 - val_loss: 0.4776 - val_accuracy: 0.7543
Epoch 4/30
166/166 [==============================] - 69s 414ms/step - loss: 0.5290 - accuracy: 0.7362 - val_loss: 0.4838 - val_accuracy: 0.7530
Epoch 5/30
166/166 [==============================] - 69s 413ms/step - loss: 0.5236 - accuracy: 0.7396 - val_loss: 0.4922 - val_accuracy: 0.7543
Epoch 6/30
166/166 [==============================] - 69s 414ms/step - loss: 0.5210 - accuracy: 0.7424 - val_loss: 0.4568 - val_accuracy: 0.7780
Epoch 7/30
166/166 [==============================] - 69s 415ms/step - loss: 0.5157 - accuracy: 0.7421 - val_loss: 0.5386 - val_a

In [18]:
model.save("gender_detection_60epochs")

In [16]:
checkpoint = ModelCheckpoint("gender_prediction_50.h5",
                             monitor="val_accuracy",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 15,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

In [17]:
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size)

Epoch 1/30
166/166 [==============================] - 64s 385ms/step - loss: 0.4797 - accuracy: 0.7716 - val_loss: 0.4647 - val_accuracy: 0.7869

Epoch 00001: val_accuracy improved from -inf to 0.78689, saving model to gender_prediction_50.h5
Epoch 2/30
166/166 [==============================] - 68s 410ms/step - loss: 0.4861 - accuracy: 0.7620 - val_loss: 0.4919 - val_accuracy: 0.7807

Epoch 00002: val_accuracy did not improve from 0.78689
Epoch 3/30
166/166 [==============================] - 68s 410ms/step - loss: 0.4803 - accuracy: 0.7700 - val_loss: 0.5526 - val_accuracy: 0.7842

Epoch 00003: val_accuracy did not improve from 0.78689
Epoch 4/30
166/166 [==============================] - 68s 412ms/step - loss: 0.4771 - accuracy: 0.7699 - val_loss: 0.4375 - val_accuracy: 0.7856

Epoch 00004: val_accuracy did not improve from 0.78689
Epoch 5/30
166/166 [==============================] - 70s 420ms/step - loss: 0.4792 - accuracy: 0.7697 - val_loss: 0.5140 - val_accuracy: 0.7869

Epoch 00